In [1]:
import os
import random
import shutil
import json
from PIL import Image
import cv2
from ultralytics import YOLO
import pandas as pd

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 


Cambio de nombre de las imágenes

In [4]:
# Número actual de imágenes
imagenes_actuales = 125

# Ruta de la carpeta original que contiene las imágenes
ruta_original = "imagenes"

# Nombre de la nueva carpeta donde se guardarán las imágenes renombradas
nueva_carpeta = "Dataset/"

# Asegurarse de que la nueva carpeta existe o se crea
if not os.path.exists(nueva_carpeta):
    os.makedirs(nueva_carpeta)

# Recorrer las imágenes en la carpeta original
for i, nombre_archivo in enumerate(os.listdir(ruta_original)):
    # Verificar si el archivo es una imagen (puedes añadir más extensiones si lo necesitas)
    if nombre_archivo.endswith(('.jpg')):
        # Definir el nuevo nombre para la imagen (ejemplo: imagen_1.jpg)
        nuevo_nombre = f"imagen_{i+1+imagenes_actuales}.jpg"

        # Ruta completa del archivo original
        ruta_original_completa = os.path.join(ruta_original, nombre_archivo)

        # Ruta completa del nuevo archivo
        ruta_nueva_completa = os.path.join(nueva_carpeta, nuevo_nombre)

        # Copiar y renombrar la imagen a la nueva carpeta
        shutil.copy(ruta_original_completa, ruta_nueva_completa)

print("Proceso completado.")


Proceso completado.


Limpieza de los directorios train, val y test para añadir imagenes nuevamente

In [10]:
# Función para limpiar los directorios de imágenes y etiquetas
def limpiar_directorios(carpeta_destino):
    # Subcarpetas para los conjuntos de datos
    subcarpetas = ['train', 'test', 'val']
    
    # Tipos de archivo que queremos eliminar
    extensiones = ['.jpg', '.png', '.txt']  # Añade aquí otras extensiones de imagen si es necesario
    
    for subcarpeta in subcarpetas:
        carpeta_imagenes = os.path.join(carpeta_destino, subcarpeta, "images")
        carpeta_labels = os.path.join(carpeta_destino, subcarpeta, "labels")
        
        # Eliminar archivos en la carpeta de imágenes
        if os.path.exists(carpeta_imagenes):
            for archivo in os.listdir(carpeta_imagenes):
                if any(archivo.endswith(ext) for ext in extensiones):
                    os.remove(os.path.join(carpeta_imagenes, archivo))
        
        # Eliminar archivos en la carpeta de etiquetas
        if os.path.exists(carpeta_labels):
            for archivo in os.listdir(carpeta_labels):
                if archivo.endswith('.txt'):
                    os.remove(os.path.join(carpeta_labels, archivo))
    
    print("Los directorios de train, test y val han sido limpiados.")

# Ejemplo de uso
carpeta_destino = "VC_P4-NUM_PLATES"  # Ruta a la carpeta principal
limpiar_directorios(carpeta_destino)

Los directorios de train, test y val han sido limpiados.


Distribución de las imágenes entre train (70%), test (20%) y val (10%).

In [7]:
# Definir las clases y el ID de la clase "matricula"
clases = ["matricula"]

# Función para convertir al formato YOLO y escribir en el archivo .txt
def convertir_a_yolo(class_id, x_min, y_min, x_max, y_max, imagen_ancho, imagen_alto, archivo_txt):
    # Calcular coordenadas y tamaño del bounding box normalizado
    x_center = ((x_min + x_max) / 2) / imagen_ancho
    y_center = ((y_min + y_max) / 2) / imagen_alto
    width = (x_max - x_min) / imagen_ancho
    height = (y_max - y_min) / imagen_alto

    # Escribir en el archivo .txt en formato YOLO
    archivo_txt.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Ruta de la carpeta original que contiene las imágenes y JSON
carpeta_original = "Dataset"

# Carpeta de destino principal
carpeta_destino = "VC_P4-NUM_PLATES"

# Subcarpetas dentro de cada subconjunto
subcarpetas = ['train', 'test', 'val']
carpeta_images = {subcarpeta: os.path.join(carpeta_destino, subcarpeta, "images") for subcarpeta in subcarpetas}
carpeta_labels = {subcarpeta: os.path.join(carpeta_destino, subcarpeta, "labels") for subcarpeta in subcarpetas}

# Asegurarse de que las carpetas de destino existen o se crean
for subcarpeta in subcarpetas:
    os.makedirs(carpeta_images[subcarpeta], exist_ok=True)
    os.makedirs(carpeta_labels[subcarpeta], exist_ok=True)

# Obtener todas las imágenes de la carpeta original
imagenes = [f for f in os.listdir(carpeta_original) if f.endswith('.jpg')]

# Mezclar las imágenes de forma aleatoria
random.shuffle(imagenes)

# Calcular cuántas imágenes irán en cada subconjunto
total_imagenes = len(imagenes)
train_size = int(0.7 * total_imagenes)
test_size = int(0.2 * total_imagenes)
val_size = total_imagenes - train_size - test_size  # Lo que queda es para val

# Dividir las imágenes en tres subconjuntos
train_imagenes = imagenes[:train_size]
test_imagenes = imagenes[train_size:train_size + test_size]
val_imagenes = imagenes[train_size + test_size:]

# Función para copiar imágenes a las carpetas 'images' y crear archivos .txt con anotaciones en 'labels'
def copiar_imagenes_con_anotaciones(imagenes, carpeta_imagenes, carpeta_labels):
    for imagen in imagenes:
        # Copiar la imagen a la subcarpeta "images"
        ruta_origen_imagen = os.path.join(carpeta_original, imagen)
        ruta_destino_imagen = os.path.join(carpeta_imagenes, imagen)
        shutil.copy(ruta_origen_imagen, ruta_destino_imagen)

        # Generar la ruta del archivo JSON correspondiente
        nombre_sin_extension = os.path.splitext(imagen)[0]  # Nombre sin extensión
        archivo_json = f"{nombre_sin_extension}.json"
        ruta_origen_json = os.path.join(carpeta_original, archivo_json)

        # Crear el archivo .txt en la carpeta "labels" y convertir el JSON a formato YOLO, si existe el JSON
        ruta_txt = os.path.join(carpeta_labels, f"{nombre_sin_extension}.txt")
        if os.path.exists(ruta_origen_json):
            with open(ruta_txt, 'w') as archivo_txt, open(ruta_origen_json, 'r') as json_file:
                datos = json.load(json_file)

                # Abrir la imagen para obtener sus dimensiones
                with Image.open(ruta_destino_imagen) as img:
                    imagen_ancho, imagen_alto = img.size

                # Recorrer los objetos anotados en el JSON y escribir en formato YOLO
                for forma in datos['shapes']:
                    if forma['label'] == 'matricula' and forma['shape_type'] == 'rectangle':
                        puntos = forma['points']
                        x_min = min(p[0] for p in puntos)
                        y_min = min(p[1] for p in puntos)
                        x_max = max(p[0] for p in puntos)
                        y_max = max(p[1] for p in puntos)

                        # Llamar a la función que convierte al formato YOLO y escribe en el archivo .txt
                        class_id = clases.index('matricula')
                        convertir_a_yolo(class_id, x_min, y_min, x_max, y_max, imagen_ancho, imagen_alto, archivo_txt)

# Copiar las imágenes y crear los archivos .txt en las carpetas correspondientes
copiar_imagenes_con_anotaciones(train_imagenes, carpeta_images['train'], carpeta_labels['train'])
copiar_imagenes_con_anotaciones(test_imagenes, carpeta_images['test'], carpeta_labels['test'])
copiar_imagenes_con_anotaciones(val_imagenes, carpeta_images['val'], carpeta_labels['val'])

print("Las imágenes y archivos .txt de anotaciones han sido divididos en train, test y val.")

Las imágenes y archivos .txt de anotaciones han sido divididos en train, test y val.


Pruebas

In [8]:
# Carga del modelo entrenado
model = YOLO('runs/detect/train2/weights/best.pt')

# Ruta del video de entrada
filename = "C0142.MP4"

# Abrir el archivo de video
cap = cv2.VideoCapture(filename)

# Obtener información sobre el video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Crear la ruta de salida para el video procesado
output_filename = 'video_con_detecciones_2.mp4'
output_path = os.path.join(os.getcwd(), output_filename)

# Crear un VideoWriter para guardar el video con detecciones
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Procesar el video frame a frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Si no hay más frames, salir del bucle

    # Realizar detecciones en el frame actual
    results = model(frame)

    # Obtener el frame anotado con las detecciones
    annotated_frame = results[0].plot()

    # Guardar el frame anotado en el archivo de video de salida
    out.write(annotated_frame)

# Liberar los recursos
cap.release()
out.release()

print(f"Video guardado en {output_path}")


0: 384x640 (no detections), 119.3ms
Speed: 11.0ms preprocess, 119.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 199.8ms
Speed: 5.0ms preprocess, 199.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 145.2ms
Speed: 6.0ms preprocess, 145.2ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 156.6ms
Speed: 5.6ms preprocess, 156.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 matricula, 113.0ms
Speed: 10.0ms preprocess, 113.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 matricula, 109.1ms
Speed: 4.0ms preprocess, 109.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 115.3ms
Speed: 6.0ms preprocess, 115.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 98.7ms
Speed: 5.0ms preprocess, 

Entrgable

In [2]:
from collections import defaultdict
import numpy as np
import cv2
import torch
import math
from ultralytics import YOLO
import easyocr
import csv
import os

# Cargar ambos modelos
model_detection = YOLO('yolo11n.pt')  # Modelo para detección de personas y coches
model_license_plate = YOLO('runs/detect/train/weights/best.pt')  # Modelo entrenado para detección de matrículas

# Inicializar EasyOCR para reconocimiento de texto
reader = easyocr.Reader(['en'])  # Puedes ajustar el idioma si lo necesitas

# Etiquetas de las clases que quieres detectar
classNames = ["person", "bicycle", "car"]

# Inicializamos contadores
counts = {'person': 0, 'car': 0, 'plate': 0}

# Archivo CSV de salida
csv_file = 'resultados_detalles.csv'
if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Tipo_objeto', 'Confianza', 'ID_Tracking', 'x1', 'y1', 'x2', 'y2',
                         'Matrícula', 'Confianza_Matrícula', 'mx1', 'my1', 'mx2', 'my2', 'Texto_Matrícula'])

# Configuración para volcar el video
output_video = 'resultados_video.mp4'
frame_width = 640  # Cambia según la resolución del video
frame_height = 480
fps = 30.0
output_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Captura desde la webcam o un video
vid = cv2.VideoCapture("videoplayback.MP4")  

frame_number = 0
while True:
    ret, img = vid.read()

    # Si el fotograma es válido
    if ret:
        frame_number += 1  # Contador de fotogramas

        # Detectar personas y coches con el primer modelo
        results = model_detection(img, classes=[0, 2])  # Clases 0 (persona) y 2 (coche)

        # Procesar detecciones del primer modelo
        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convertir a enteros
                cls = int(box.cls[0])
                confidence = float(box.conf[0])

                object_type = classNames[cls]

                # Contar detecciones
                if object_type == "person":
                    counts['person'] += 1
                elif object_type == "car":
                    counts['car'] += 1

                # Tracking ID (si existe)
                track_id = str(int(box.id[0].tolist())) if box.id is not None else ''

                if object_type == "car":
                    # Si es un coche, recortar la región detectada del coche
                    car_img = img[y1:y2, x1:x2]

                    # Pasar la imagen del coche al modelo de matrículas
                    plate_results = model_license_plate(car_img)

                    # Dibujar el recuadro de la matrícula en la imagen original
                    for plate in plate_results:
                        plate_boxes = plate.boxes
                        for pbox in plate_boxes:
                            px1, py1, px2, py2 = pbox.xyxy[0]
                            px1, py1, px2, py2 = int(px1), int(py1), int(px2), int(py2)

                            # Recortar la matrícula detectada
                            plate_img = car_img[py1:py2, px1:px2]

                            # Mejorar la imagen (opcional) para el OCR
                            gray_plate = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
                            enhanced_plate = cv2.bilateralFilter(gray_plate, 11, 17, 17)  # Mejora la nitidez

                            # Usar EasyOCR para leer la matrícula
                            result = reader.readtext(enhanced_plate, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')

                            # Si se ha leído texto, mostrarlo y guardarlo en el CSV
                            if result:
                                counts['plate'] += 1  # Contar matrículas
                                for (bbox, text, prob) in result:
                                    # Coordenadas en orden 
                                    (top_left, top_right, bottom_right, bottom_left) = bbox
                                    print(f'\nTexto: {text}\nProbabilidad: {prob:.2f}')
                                    # Guardar matrícula y probabilidad en el CSV
                                    with open(csv_file, mode='a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow([frame_number, 'car', f'{confidence:.2f}', track_id,
                                                         x1, y1, x2, y2, text, f'{prob:.2f}',
                                                         px1, py1, px2, py2, text])

                                    # Dibujar el texto en la imagen
                                    cv2.putText(img, text, (x1+px1, y1+py1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                                # Dibujar el contenedor de la matrícula
                                cv2.rectangle(img, (x1+px1, y1+py1), (x1+px2, y1+py2), (0, 255, 0), 3)
                                cv2.putText(img, "plate", (x1+px1, y1+py1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Dibujar el contenedor para personas y coches
                color = (0, 0, 255) if object_type == "person" else (255, 0, 0)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, object_type, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # Mostrar el fotograma con las detecciones
        cv2.imshow('Video', img)

        # Guardar el fotograma en el vídeo de salida
        output_writer.write(img)

    # Salir si se presiona 'Esc'
    if cv2.waitKey(20) & 0xFF == 27:
        break

# Liberar recursos
vid.release()
output_writer.release()
cv2.destroyAllWindows()

# Mostrar conteo final
print(f'Personas detectadas: {counts["person"]}')
print(f'Coches detectados: {counts["car"]}')
print(f'Matrículas detectadas: {counts["plate"]}')


100%|██████████| 5.35M/5.35M [00:00<00:00, 5.91MB/s]



0: 384x640 (no detections), 41.0ms
Speed: 2.5ms preprocess, 41.0ms inference, 21.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.5ms
Speed: 3.5ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.0ms
Speed: 1.5ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.5ms
Speed: 2.0ms preprocess, 9.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.5ms
Speed: 1.5ms preprocess, 10.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.0ms
Speed: 2.0ms preprocess, 15.0ms in